In [31]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Resizing, Rescaling, Dense, Conv2D, MaxPooling2D, Flatten, RandomFlip, RandomContrast, RandomRotation, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import Sequential
from tensorflow.image import random_flip_up_down, random_flip_left_right, random_contrast, random_brightness, rot90
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import random
import numpy as np
from tensorflow.keras.utils import image_dataset_from_directory
from typing import Literal

In [2]:
# Set seed for reproduceability
SEED = 42

# Size to convert images to (pixels)
RESIZE_HEIGHT = 750
RESIZE_WIDTH = 750

In [3]:
# Load photo data
train_images = image_dataset_from_directory(directory = 'data/Training', color_mode="grayscale", label_mode = 'binary', batch_size=32)
test_images = image_dataset_from_directory(directory = 'data/Testing', color_mode="grayscale", label_mode = 'binary', batch_size=32)

Found 5712 files belonging to 2 classes.
Found 1311 files belonging to 2 classes.


In [4]:
# Images are not standardized. Need to standardize prior to modelling
def resize_images(image, label):
    image = tf.image.resize_with_crop_or_pad(image, target_height = RESIZE_HEIGHT, target_width = RESIZE_WIDTH)
    return image, label

train_images = train_images.map(resize_images)
test_images = test_images.map(resize_images)

In [5]:
# Runtime optimization
AUTOTUNE = tf.data.AUTOTUNE 
train_images = train_images.prefetch(buffer_size = AUTOTUNE)
test_images = test_images.prefetch(buffer_size = AUTOTUNE)

In [25]:
# Custom classifier method
'''
Inputs -> Preprocessing -> Augmentation (training) -> Convolution ->
Max Pooling -> Convolution -> Max Pooling -> Flatten -> Dense ->
Dense -> Dropout -> Dense (classifier)
'''
class v1TumorClassifier(tf.keras.Model):
    def __init__(self):
        super().__init__()

        # Standardization layer that resizes to and scales  
        self.standardization_layer = Sequential([
            Rescaling(1 / 255.)
        ])
        self.augmentation_layer = Sequential([
            tf.keras.Input(shape = (RESIZE_HEIGHT, RESIZE_WIDTH, 1)),
            RandomRotation(factor = (-0.3, 0.3)),
            RandomFlip("horizontal_and_vertical"),
            RandomContrast(factor = (-0.3, 0.3))
        ])

        self.conv1 = Conv2D(16, (3,3), activation = 'relu')
        self.pool1 = MaxPooling2D()
        self.conv2 = Conv2D(32, (3,3), activation = 'relu')
        self.pool2 = MaxPooling2D()

        self.dropout = Dropout(0.3)
        self.flatten = Flatten()

        self.dense1 = Dense(128, activation = 'relu')
        self.dense2 = Dense(64, activation = 'relu')

        self.classifier = Dense(1, activation = 'sigmoid')

    def call(self, inputs, training = False):
        x = self.standardization_layer(inputs)
        x = self.augmentation_layer(x)
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dropout(x, training = training)
        output = self.classifier(x)
        return output


In [26]:
custom_model = v1TumorClassifier()

custom_model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy','recall']  
)

custom_model_history = custom_model.fit(train_images, validation_data = test_images, epochs = 5, batch_size = 256)

Epoch 1/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.7556 - loss: 0.5685 - recall: 0.8758 - val_accuracy: 0.8162 - val_loss: 0.3967 - val_recall: 0.9724
Epoch 2/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.9183 - loss: 0.2386 - recall: 0.9634 - val_accuracy: 0.8436 - val_loss: 0.3664 - val_recall: 0.9570
Epoch 3/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.9412 - loss: 0.1726 - recall: 0.9674 - val_accuracy: 0.8612 - val_loss: 0.3715 - val_recall: 0.9547
Epoch 4/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.9514 - loss: 0.1528 - recall: 0.9731 - val_accuracy: 0.9069 - val_loss: 0.2438 - val_recall: 0.9172
Epoch 5/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 196s 1s/step - accuracy: 0.9553 - loss: 0.1366 - recall: 0.9684 - val_accuracy: 0.9108 - val_loss: 0.2222 - val_recall: 0.9470


In [34]:
# Transfer learning method(s): ResNet and DenseNet
from tensorflow.keras.applications import ResNet50, DenseNet121

V2_IMAGE_SIZE = (256,256)

# Load photo data
v2_train_images = image_dataset_from_directory(directory = 'data/Training', color_mode="rgb", label_mode = 'binary', batch_size=32)
v2_test_images = image_dataset_from_directory(directory = 'data/Testing', color_mode="rgb", label_mode = 'binary', batch_size=32)

def v2_resize_images(image, label):
    image = tf.image.resize(image, method='bilinear', size = V2_IMAGE_SIZE)
    return image, label

train_images = train_images.map(v2_resize_images)
test_images = test_images.map(v2_resize_images)



Found 5712 files belonging to 2 classes.
Found 1311 files belonging to 2 classes.


In [ ]:
# Transfer learning classifier method
'''
Inputs -> Preprocessing -> Augmentation (training) -> Pretrained Base ->
Global Max/Avg Pooling -> Dense -> Dropout -> Dense (classifier)
'''
class v2TumorClassifier(tf.keras.Model):
    def __init__(self, base_model = Literal['resnet','densenet'], pool_type = Literal['max','avg']):
        super().__init__()

        # Base model assignment
        if base_model == 'resnet':
            self.base_model = ResNet50(
                weights = 'imagenet',
                include_top = False,
                input_shape = (V2_IMAGE_SIZE[0], V2_IMAGE_SIZE[1], 3)
            )

        if base_model == 'densenet':
            self.base_model = DenseNet121(
                weights = 'imagenet',
                include_top = False,
                input_shape = (V2_IMAGE_SIZE[0], V2_IMAGE_SIZE[1], 3)
            )

        # Base model should not be getting retrained
        self.base_model.trainable = False

        # Standardization layer that resizes to and scales  
        self.standardization_layer = Sequential([
            Rescaling(1 / 255.)
        ])
        # Augmentation layer that augments data
        self.augmentation_layer = Sequential([
            tf.keras.Input(shape = (V2_IMAGE_SIZE[0], V2_IMAGE_SIZE[1], 3)),
            RandomRotation(factor = (-0.3, 0.3)),
            RandomFlip("horizontal_and_vertical"),
            RandomContrast(factor = (-0.3, 0.3))
        ])

        # Global pooling layer
        if pool_type == 'max':
            self.pool_layer = GlobalMaxPooling2D()
        if pool_type == 'avg':
            self.pool_layer = GlobalAveragePooling2D()

        # Dropout, dense, classifier
        self.dropout = Dropout(0.3)
        self.dense = Dense(32, activation = 'relu')
        self.classifier = Dense(1, activation = 'sigmoid')
    
    def call(self, inputs, training = False):
        x = self.standardization_layer(inputs)
        x = self.augmentation_layer(x)
        x = self.base_model(x, training = training)
        x = self.pool_layer(x)
        x = self.dense(x)
        x = self.dropout(x, training = training)
        output = self.classifier(x)
        return output



In [36]:
resnet_1 = v2TumorClassifier(base_model = 'resnet', pool_type = 'avg')

resnet_1.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy','recall']
)

resnet_1_history = resnet_1.fit(v2_train_images, validation_data = v2_test_images, epochs = 5, batch_size = 256)

Epoch 1/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 101s 551ms/step - accuracy: 0.8452 - loss: 0.4053 - recall: 0.9551 - val_accuracy: 0.7452 - val_loss: 0.5945 - val_recall: 0.9901
Epoch 2/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 98s 548ms/step - accuracy: 0.8999 - loss: 0.3005 - recall: 0.9793 - val_accuracy: 0.7872 - val_loss: 0.4732 - val_recall: 0.9757
Epoch 3/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 97s 541ms/step - accuracy: 0.9056 - loss: 0.2930 - recall: 0.9792 - val_accuracy: 0.7628 - val_loss: 0.5361 - val_recall: 0.9868
Epoch 4/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 96s 538ms/step - accuracy: 0.9049 - loss: 0.2769 - recall: 0.9800 - val_accuracy: 0.7536 - val_loss: 0.5436 - val_recall: 0.9857
Epoch 5/5
179/179 ━━━━━━━━━━━━━━━━━━━━ 97s 544ms/step - accuracy: 0.9079 - loss: 0.2660 - recall: 0.9789 - val_accuracy: 0.7879 - val_loss: 0.4734 - val_recall: 0.9746
